In [1]:
from model.Corpus import Corpus
from model.Model import Model
from model.Embeddings import Embeddings
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

from pprint import pprint
import csv
from scrapers.ScraperResult import ScraperResult
from page_processors import *


global_hyperparameters = {
    "embedding_dim": 100, # Dimensions in the GloVe embedding
    "max_length": 60, # Enforced max and min length of an example
    "trunc_type": "post",
    "padding_type": "post",
    "oov_tok": "<OOV>",
    "test_portion": 0.1
}

embedding_dim = 100
max_length = 60

trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
test_portion = 0.1


In [2]:
# should tokenize the labels; there aren't 1191 unique ones
num_epochs_i = 6
training_size_i = 89500

ingredients_hyperparameters = {
    "num_epochs": 6,
    "training_size": 89500
}

ingredient_corpus = Corpus(
    "training_data/augmented_classified_ingredients.csv",
    global_hyperparameters,
    ingredients_hyperparameters
).initialize()

ingredient_embeddings = Embeddings(
    "glove.6B.100d.txt",
    #embedding_dim,
    ingredient_corpus,
    global_hyperparameters
).activate()

ingredient_classifier = Model(
    ingredient_corpus,
    ingredient_embeddings,
    global_hyperparameters,
    ingredients_hyperparameters,
    #max_length,
    #num_epochs_i,
    1191
).run()


Epoch 1/6
2518/2518 [==============================] - 72s 29ms/step - loss: 1.3860 - accuracy: 0.7246 - val_loss: 0.1891 - val_accuracy: 0.9474
Epoch 2/6
2518/2518 [==============================] - 75s 30ms/step - loss: 0.1961 - accuracy: 0.9464 - val_loss: 0.1110 - val_accuracy: 0.9686
Epoch 3/6
2518/2518 [==============================] - 73s 29ms/step - loss: 0.1242 - accuracy: 0.9635 - val_loss: 0.0779 - val_accuracy: 0.9754
Epoch 4/6
2518/2518 [==============================] - 73s 29ms/step - loss: 0.1049 - accuracy: 0.9676 - val_loss: 0.0647 - val_accuracy: 0.9782
Epoch 5/6
2518/2518 [==============================] - 73s 29ms/step - loss: 0.0910 - accuracy: 0.9715 - val_loss: 0.0570 - val_accuracy: 0.9808
Epoch 6/6
2518/2518 [==============================] - 73s 29ms/step - loss: 0.0802 - accuracy: 0.9738 - val_loss: 0.0727 - val_accuracy: 0.9777


In [5]:
num_epochs_r = 4
training_size_r = 76000

recipes_hyperparameters = {
    "num_epochs": 4,
    "training_size": 76000
}

recipe_corpus = Corpus(
    "training_data/augmented_training_labels.csv",
    global_hyperparameters,
    recipes_hyperparameters
).initialize()

recipe_embeddings = Embeddings(
    "glove.6B.100d.txt",
    recipe_corpus,
    global_hyperparameters
).activate()

recipe_classifier = Model(
    recipe_corpus,
    recipe_embeddings,
    global_hyperparameters,
    recipes_hyperparameters,
    5
).run()


Epoch 1/4
2138/2138 [==============================] - 53s 25ms/step - loss: 0.0644 - accuracy: 0.9789 - val_loss: 0.0341 - val_accuracy: 0.9883
Epoch 2/4
2138/2138 [==============================] - 55s 26ms/step - loss: 0.0268 - accuracy: 0.9916 - val_loss: 0.0255 - val_accuracy: 0.9918
Epoch 3/4
2138/2138 [==============================] - 53s 25ms/step - loss: 0.0205 - accuracy: 0.9932 - val_loss: 0.0192 - val_accuracy: 0.9941
Epoch 4/4
2138/2138 [==============================] - 53s 25ms/step - loss: 0.0181 - accuracy: 0.9939 - val_loss: 0.0241 - val_accuracy: 0.9933


In [76]:
total_payload = []

new_links = [
    "https://www.bbcgoodfood.com/recipes/espresso-martini"
]

with open('newlabels.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    for link in new_links:
        print(link)
        try:
            data = process_page(
                link,
                writer,
                recipe_classifier,
                recipe_corpus,
                max_length,
                padding_type,
                trunc_type
            )
            this_recipe = ScraperResult(
                link.split("/")[-1],
                list(map(lambda x: x[0], lasso_inward(data,4,6,1,7)[0])),
                list(map(lambda x: x[0], lasso_inward(data,4,6,2,7)[0])),
                list(map(lambda x: x[0], lasso_inward(data,4,6,3,7)[0])),
                list(map(lambda x: x[0], lasso_inward(data,4,6,4,7)[0]))
            )
            this_payload = this_recipe.get_payload()
            total_payload.append(this_recipe.get_payload())

        except:
            print("link " + link + " could not be opened")




https://www.bbcgoodfood.com/recipes/espresso-martini
utf-8


In [77]:
tst_snt = total_payload[0]["ingredients"]
tst_snt = ingredient_corpus.tokenizer.texts_to_sequences(tst_snt)
tst_snt = pad_sequences(
    tst_snt, 
    maxlen=max_length, 
    padding=padding_type, 
    truncating=trunc_type
)
pred = ingredient_classifier.model.predict(tst_snt)

for j in range(len(total_payload)):
    for i in range(len(total_payload[j]["ingredients"])):
        pred_id = np.argmax(pred[i])
        matched_ing = list(filter(lambda x: x[1] == pred_id,ingredient_cache))[0]
        total_payload[j]["ingredients"][i] = [total_payload[j]["ingredients"][i],matched_ing[0],pred_id]

    pprint(total_payload[j]["ingredients"])



NameError: name 'ingredient_cache' is not defined

In [ ]:
from database.Query import Query
ingredient_cache = Query("select singular_name, id from ingredient;").get_data()

In [ ]:
from scrapers.CocktailLinkScraper import CocktailLinkScraper
cs = CocktailLinkScraper()

ab = cs.alton_brown("scraper_working_data/ab-cocktails.html")



In [ ]:
print(ab)

In [62]:
link = "https://www.seriouseats.com/recipes/2008/01/cocktails-manhattan-recipe.html"
data = process_page(
    link,
    recipe_classifier,
    recipe_corpus,
    max_length,
    padding_type,
    trunc_type
)

this_recipe = ScraperResult(
    link.split("/")[-1],
    list(map(lambda x: x[0], lasso_inward(data,4,6,1,7)[0])),
    list(map(lambda x: x[0], lasso_inward(data,4,6,2,7)[0])),
    list(map(lambda x: x[0], lasso_inward(data,4,6,3,7)[0])),
    list(map(lambda x: x[0], lasso_inward(data,4,6,4,7)[0]))
)
#print(data)
print(this_recipe.get_payload())

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


utf-8
{'title': 'cocktails-manhattan-recipe.html', 'steps': [''], 'ingredients': [''], 'yield': [''], 'timing': ['']}


In [63]:
print(data)

[['�\x08\x00�H�`\x02���v�F���m\x18dJ}\x1a�1 u��\x1a�/\x16�V#c{N\x03|X��յa\x19�,9 Z.\x16A\x18�9��\x1b,#jǑ9\x0e��0\x18�؍=:|m�3;�m� \x06���v=��\x05% �\x11�H�C̓\x1a/����F\x07څK/�\x11\x05�K\u05c9g\x03�^�cj�\x17���\x1b��gDcۣ\x03ˬ�.y�\x7fNf!� 4$Hԯ�.//� µ�!\x14�\x1a��^�n��\x02�ȘK� ^��Q�HH��6���w�A������Ǘn Ӱ?�CGA>Z��vx}��ᔞ�s[R��j���`�tV���,h\x18_\x0f�`�g 4 Tꬥ>�9۬C�q�.b 7�\x15\x08�3 Jƞ E�H��dn;�\x00�3\x12^S�D���䂆�`��a��@�\x13��i)��h�w�^_\x06�\x13)�$�\x11�\x04� W>Ri���\x13��\\e�k�\x16��7\x1bM#e/`��k�Ҏ�s�ՠX�ʲ\x1am��b� �e�}"Bw�o�$\x133��\x01f�c��\x14�!�Y��g\x02Z~�L�`��Q�N�}\x05tqh`,B�ӿ_�\x13��8�,C��\x05�\x07��� D\x0e�yΣ�Q�%�\x07֚�g\x06�*����\x07�\x13�gtn�A 8�tQ\x08Aa�7�����\x194��hb�*�d �_\x00^CxY� �\x01\x14� �k� *���,\\\x18�\x10��o�V^��� �О߱�5 L���\x04\x05Ú\x18e 6Ȭ�)�h�lW�|�ؐ��?z���H\x14_{4\x12K^��s�\x01 GQmʀ�s�7���ŠQ\x07�a 5�b���@�i\x1bl�pcx�c�� nO��\x18gK��!\x0e�?�������;����西XƯi< �C> wv���`���<7������\x0f\x17vH� �D ����8�n�� ŷ���֪��ɥ\x00���hzl�\x11��{Sǥ�UF\x15�� x�� �\\ϩ��� d�xV���V��~� B�1�yE�

In [75]:
import re
import urllib
from bs4 import BeautifulSoup, Comment
from tensorflow.keras.preprocessing.sequence import pad_sequences

url = "https://www.seriouseats.com/recipes/2008/01/cocktails-manhattan-recipe.html"

#soup = BeautifulSoup(open(url, 'r'),"html.parser",from_encoding="iso-8859-1")




user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,}

request = urllib.request.Request(url,None,headers)
f = urllib.request.urlopen(request)
file = f.read()
soup = BeautifulSoup(file,'html.parser')



Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
